In [ ]:
%load_ext autoreload
%autoreload 2

from MySpotify import MySpotify

SpotifyModule = MySpotify("data.zip")

In [ ]:
SpotifyModule.convert_files()

In [ ]:
SpotifyModule.PreProcess_Data()

In [ ]:
TopTracksNum = 250
best_tracks = SpotifyModule.get_Top_Tracks(TopTracksNum)

In [ ]:
TopTracksByGenreNum = 100
genre_list = ["Rock", "Rap", "Jazz", "Electronic", "Pop", "Blues", "Country", "Reggae", "New Age"]
for genre in genre_list:
    best_tracks_by_genre = SpotifyModule.Get_TopTracks_By_Genre(TopTracksByGenreNum, genre, False)
    input("Press Enter to continue to the next genre...")

In [ ]:
from pyarrow.parquet import ParquetFile
import pandas as pd

pf = ParquetFile("data/parquet/Merged_tracks_data.parquet")

df : pd.DataFrame = pf.read().to_pandas()